# Set up the class fundementals 

In [1]:
# import os, sys
# import collections
import numpy as _np
# import matplotlib.markers as markers
# import matplotlib.pyplot as plt
# import timeit
# import collections
# from scipy.stats import median_abs_deviation as mad
# import multiprocessing
# import cProfile
# import itertools
from numba import jit as _jit
from numba import njit as _njit
# import warnings
# warnings.filterwarnings('ignore') # this is just to hide all the warnings
# import rpy2.robjects as robjects
# import matplotlib.pyplot as plt # change font globally to Times 
# plt.style.use('ggplot')
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Times New Roman",
#     "font.sans-serif": ["Times New Roman"],
#     "font.size": 12})
import multiprocess as _mp

# os.chdir(sys.path[0]) # ensure working direcotry is set same as the file

In [2]:
######################################  some SCAD and MCP things  #######################################
@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def soft_thresholding(x, lambda_):
    '''
    To calculate soft-thresholding mapping of a given ONE-DIMENSIONAL tensor, BESIDES THE FIRST TERM (so beta_0 will not be penalized). 
    This function is to be used for calculation involving L1 penalty term later. 
    '''
    return _np.hstack((_np.array([x[0]]), _np.where(_np.abs(x[1:])>lambda_, x[1:] - _np.sign(x[1:])*lambda_, 0)))

soft_thresholding(_np.random.rand(20),3.1)

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SCAD(x, lambda_, a=3.7):
    '''
    To calculate SCAD penalty value;
    #x can be a multi-dimensional tensor;
    lambda_, a are scalars;
    Fan and Li suggests to take a as 3.7 
    '''
    # here I notice the function is de facto a function of absolute value of x, therefore take absolute value first to simplify calculation 
    x = _np.abs(x)
    temp = _np.where(x<=lambda_, lambda_*x, _np.where(x<a*lambda_, (2*a*lambda_*x - x**2 - lambda_**2)/(2*(a - 1)), lambda_**2 * (a+1)/2))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SCAD_grad(x, lambda_, a=3.7):
    '''
    To calculate the gradient of SCAD wrt. input x; 
    #x can be a multi-dimensional tensor. 
    '''
    # here decompose x to sign and its absolute value for easier calculation
    sgn = _np.sign(x)
    x = _np.abs(x)
    temp = _np.where(x<=lambda_, lambda_*sgn, _np.where(x<a*lambda_, (a*lambda_*sgn-sgn*x)/(a-1), 0))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def MCP(x, lambda_, gamma):
    '''
    To calculate MCP penalty value; 
    #x can be a multi-dimensional tensor. 
    '''
    # the function is a function of absolute value of x 
    x = _np.abs(x)
    temp = _np.where(x<=gamma*lambda_, lambda_*x - x**2/(2*gamma), .5*gamma*lambda_**2)
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def MCP_grad(x, lambda_, gamma):
    '''
    To calculate MCP gradient wrt. input x; 
    #x can be a multi-dimensional tensor. 
    '''
    temp = _np.where(_np.abs(x)<gamma*lambda_, lambda_*_np.sign(x)-x/gamma, _np.zeros_like(x))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SCAD_concave(x, lambda_, a=3.7):
    '''
    The value of concave part of SCAD penalty; 
    #x can be a multi-dimensional tensor. 
    '''
    x = _np.abs(x)
    temp = _np.where(x<=lambda_, 0., _np.where(x<a*lambda_, (lambda_*x - (x**2 + lambda_**2)/2)/(a-1), (a+1)/2*lambda_**2 - lambda_*x))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def SCAD_concave_grad(x, lambda_, a=3.7):
    '''
    The gradient of concave part of SCAD penalty wrt. input x; 
    #x can be a multi-dimensional tensor. 
    '''
    sgn = _np.sign(x)
    x = _np.abs(x)
    temp = _np.where(x<=lambda_, 0., _np.where(x<a*lambda_, (lambda_*sgn-sgn*x)/(a-1), -lambda_*sgn))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def MCP_concave(x, lambda_, gamma):
    '''
    The value of concave part of MCP penalty; 
    #x can be a multi-dimensional tensor. 
    '''
    # similiar as in MCP
    x = _np.abs(x)
    temp = _np.where(x<=gamma*lambda_, -(x**2)/(2*gamma), (gamma*lambda_**2)/2 - lambda_*x)
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 

@_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def MCP_concave_grad(x, lambda_, gamma):
    '''
    The gradient of concave part of MCP penalty wrt. input x; 
    #x can be a multi-dimensional tensor. 
    '''
    temp = _np.where(_np.abs(x) < gamma*lambda_, -x/gamma, -lambda_*_np.sign(x))
    temp[0] = 0. # this is to NOT penalize intercept beta later 
    return temp 


# Implementation

In [3]:
# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def _memmap_update_smooth_grad_convex_LM_parallel(N, p, X, beta_md, y, _dtype, _order, chunck_size=50000):
    '''
    Update the gradient of the smooth convex objective component.
    '''
    _itemsize = _np.dtype(_dtype).itemsize
    # first calcualte _=X@beta_md-y
    if _order == "F":
        def __parallel_plus(_ind):
            import numpy as np
            __ = _np.zeros(N)
            for j in _ind:
                _X = _np.memmap(X, dtype=_dtype, mode='r', offset=j*_itemsize*N, shape = (N,))
                __ += _X*beta_md[j]
            return __
        # multiprocessing starts here
        ind = _np.arange(p)
        n_slices = _np.ceil(len(ind)/chunck_size)
        with _mp.Pool(_mp.cpu_count()) as pl:
            _ = pl.map(__parallel_plus, _np.array_split(ind, n_slices))
        _ = _np.array(_).sum(0)
    elif _order == "C":
        def __parallel_assign(_ind):
            import numpy as np
            k=0
            __ = _np.zeros(len(_ind))
            for j in _ind:
                _X = _np.memmap(X, dtype=_dtype, mode='r', offset=j*_itemsize*p, shape = (p,))
                __[k] = _X@beta_md
                k += 1
            return __
        # multiprocessing starts here
        ind = _np.arange(N)
        n_slices = _np.ceil(len(ind)/chunck_size)
        with _mp.Pool(_mp.cpu_count()) as pl:
            _ = pl.map(__parallel_assign, _np.array_split(ind, n_slices))
        _ = _np.hstack(_)
    _ -= y
    # then calculate _XTXbeta = X.T@X@beta_md = X.T@_
    if _order == "F":
        def __parallel_assign(_ind):
            import numpy as np
            k=0
            __ = _np.zeros(len(_ind))
            for j in _ind:
                _X = _np.memmap(X, dtype=_dtype, mode='r', offset=j*_itemsize*N, shape = (N,))
                __[k] = _X@_
                k += 1
            return __
        # multiprocessing starts here
        ind = _np.arange(p)
        n_slices = _np.ceil(len(ind)/chunck_size)
        with _mp.Pool(_mp.cpu_count()) as pl:
            _XTXbeta = pl.map(__parallel_assign, _np.array_split(ind, n_slices))
        _XTXbeta = _np.hstack(_XTXbeta)
    elif _order == "C":
        def __parallel_plus(_ind):
            import numpy as np
            __ = _np.zeros(p)
            for j in _ind:
                _X = _np.memmap(X, dtype=_dtype, mode='r', offset=j*_itemsize*p, shape = (p,))
                __ += _X*_[j]
            return __
        # multiprocessing starts here
        ind = _np.arange(N)
        n_slices = _np.ceil(len(ind)/chunck_size)
        with _mp.Pool(_mp.cpu_count()) as pl:
            _XTXbeta = pl.map(__parallel_plus, _np.array_split(ind, n_slices))
        _XTXbeta = _np.array(_XTXbeta).sum(0)
    del _
    return 1/N*_XTXbeta

# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def _memmap_update_smooth_grad_SCAD_LM_parallel(N, p, X, beta_md, y, _lambda, a, _dtype, _order, chunck_size):
    '''
    Update the gradient of the smooth objective component for SCAD penalty.
    '''
    return _memmap_update_smooth_grad_convex_LM_parallel(N=N, p=p, X=X, beta_md=beta_md, y=y, _dtype=_dtype, _order=_order, chunck_size=chunck_size) + SCAD_concave_grad(x=beta_md, lambda_=_lambda, a=a)

# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def _memmap_update_smooth_grad_MCP_LM_parallel(N, p, X, beta_md, y, _lambda, gamma, _dtype, _order, chunck_size):
    '''
    Update the gradient of the smooth objective component for MCP penalty.
    '''
    return _memmap_update_smooth_grad_convex_LM_parallel(N=N, p=p, X=X, beta_md=beta_md, y=y, _dtype=_dtype, _order=_order, chunck_size=chunck_size) + MCP_concave_grad(x=beta_md, lambda_=_lambda, gamma=gamma)


# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def memmap_lambda_max_LM_parallel(X, y, N, p, _dtype, _order, chunck_size=50000):
    """
    Calculate the lambda_max, i.e., the minimum lambda to nullify all penalized betas.
    """
#     X_temp = X.copy()
#     X_temp = X_temp[:,1:]
#     X_temp -= _np.mean(X_temp,0).reshape(1,-1)
#     X_temp /= _np.std(X_temp,0)
#     y_temp = y.copy()
#     y_temp -= _np.mean(y)
#     y_temp /= _np.std(y)

    grad_at_0 = _memmap_update_smooth_grad_convex_LM_parallel(N=N, p=p, X=X, beta_md=_np.zeros(p), y=y, _dtype=_dtype, _order=_order, chunck_size=chunck_size)
    lambda_max = _np.linalg.norm(grad_at_0[1:], ord=_np.infty)
    return lambda_max



# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def memmap_UAG_LM_SCAD_MCP_parallel(design_matrix, outcome, N, p, L_convex, _dtype, _order, beta_0 = _np.ones(1), tol=1e-2, maxit=500, _lambda=.5, penalty="SCAD", a=3.7, gamma=2., chunck_size=50000):
    '''
    Carry out the optimization for penalized LM for a fixed lambda.
    '''
    X = design_matrix
    y = outcome
    _itemsize = _np.dtype(_dtype).itemsize
    if _np.all(beta_0==_np.ones(1)):        
        if _order == "F":
            def __parallel_assign(_ind):
                import numpy as np
                k=0
                __ = _np.zeros(len(_ind))
                for j in _ind:
                    _X = _np.memmap(X, dtype=_dtype, mode='r', offset=j*_itemsize*N, shape = (N,))
                    __[k] = _X@y
                    k += 1
                return __
            # multiprocessing starts here
            ind = _np.arange(p)
            n_slices = _np.ceil(len(ind)/chunck_size)
            with _mp.Pool(_mp.cpu_count()) as pl:
                _XTy = pl.map(__parallel_assign, _np.array_split(ind, n_slices))
            _XTy = _np.hstack(_XTy)
        elif _order == "C":
            def __parallel_plus(_ind):
                import numpy as np
                __ = _np.zeros(p)
                for j in _ind:
                    _X = _np.memmap(X, dtype=_dtype, mode='r', offset=j*_itemsize*p, shape = (p,))
                    __ += _X*y[j]
                return __
            # multiprocessing starts here
            ind = _np.arange(N)
            n_slices = _np.ceil(len(ind)/chunck_size)
            with _mp.Pool(_mp.cpu_count()) as pl:
                _XTy = pl.map(__parallel_plus, _np.array_split(ind, n_slices))
            _XTy = _np.array(_XTy).sum(0)
        beta = _np.sign(_XTy)
    else:
        beta = beta_0
    # passing other parameters
    smooth_grad = _np.ones(p)
    beta_ag = beta.copy()
    beta_md = beta.copy()
    k = 0
    converged = False
    opt_alpha = 1.
    old_speed_norm = 1.
    speed_norm = 1.
    restart_k = 0
    
    if penalty == "SCAD":
#         L = _np.max(_np.array([L_convex, 1./(a-1)]))
        L = _np.linalg.norm(_np.array([L_convex, 1./(a-1)]), ord=_np.infty)
        opt_beta = .99/L
        while ((not converged) or (k<3)) and k <= maxit:
            k += 1
            if old_speed_norm > speed_norm and k - restart_k>=3: # in this case, restart
                opt_alpha = 1. # restarting
                restart_k = k # restarting
            else: # restarting
                opt_alpha = 2/(1+(1+4./opt_alpha**2)**.5) #parameter settings based on minimizing Ghadimi and Lan's rate of convergence error upper bound 
            opt_lambda = opt_beta/opt_alpha #parameter settings based on minimizing Ghadimi and Lan's rate of convergence error upper bound
            beta_md_old = beta_md.copy() # restarting
            beta_md = (1-opt_alpha)*beta_ag + opt_alpha*beta
            old_speed_norm = speed_norm # restarting
            speed_norm = _np.linalg.norm(beta_md - beta_md_old, ord=2) # restarting
            converged = (_np.linalg.norm(beta_md - beta_md_old, ord=_np.infty) < tol)
            smooth_grad = _memmap_update_smooth_grad_SCAD_LM_parallel(N=N, p=p, X=X, beta_md=beta_md, y=y, _lambda=_lambda, a=a, _dtype=_dtype, _order=_order, chunck_size=chunck_size)
            beta = soft_thresholding(x=beta - opt_lambda*smooth_grad, lambda_=opt_lambda*_lambda)
            beta_ag = soft_thresholding(x=beta_md - opt_beta*smooth_grad, lambda_=opt_beta*_lambda)
#             converged = _np.all(_np.max(_np.abs(beta_md - beta_ag)/opt_beta) < tol).item()
#             converged = (_np.linalg.norm(beta_md - beta_ag, ord=_np.infty) < (tol*opt_beta))
    else:
#         L = _np.max(_np.array([L_convex, 1./(gamma)]))
        L = _np.linalg.norm(_np.array([L_convex, 1./(gamma)]), ord=_np.infty)
        opt_beta = .99/L
        while ((not converged) or (k<3)) and k <= maxit:
            k += 1
            if old_speed_norm > speed_norm and k - restart_k>=3: # in this case, restart
                opt_alpha = 1. # restarting
                restart_k = k # restarting
            else: # restarting
                opt_alpha = 2/(1+(1+4./opt_alpha**2)**.5) #parameter settings based on minimizing Ghadimi and Lan's rate of convergence error upper bound 
            opt_lambda = opt_beta/opt_alpha #parameter settings based on minimizing Ghadimi and Lan's rate of convergence error upper bound
            beta_md_old = beta_md.copy() # restarting
            beta_md = (1-opt_alpha)*beta_ag + opt_alpha*beta
            old_speed_norm = speed_norm # restarting
            speed_norm = _np.linalg.norm(beta_md - beta_md_old, ord=2) # restarting
            converged = (_np.linalg.norm(beta_md - beta_md_old, ord=_np.infty) < tol)
            smooth_grad = _memmap_update_smooth_grad_MCP_LM_parallel(N=N, p=p, X=X, beta_md=beta_md, y=y, _lambda=_lambda, gamma=gamma, _dtype=_dtype, _order=_order, chunck_size=chunck_size)
            beta = soft_thresholding(x=beta - opt_lambda*smooth_grad, lambda_=opt_lambda*_lambda)
            beta_ag = soft_thresholding(x=beta_md - opt_beta*smooth_grad, lambda_=opt_beta*_lambda)
#             converged = _np.all(_np.max(_np.abs(beta_md - beta_ag)/opt_beta) < tol).item()
#             converged = (_np.linalg.norm(beta_md - beta_ag, ord=_np.infty) < (tol*opt_beta))
    return k, beta_md


# @_jit(nopython=True, cache=True, parallel=True, fastmath=True, nogil=True)
def memmap_solution_path_LM_parallel(design_matrix, outcome, lambda_, L_convex, N, p, beta_0 = _np.ones(1), tol=1e-2, maxit=500, penalty="SCAD", a=3.7, gamma=2., _dtype='float32', _order="F", chunck_size=50000):
    '''
    Carry out the optimization for the solution path without the strong rule.
    '''
    beta_mat = _np.zeros((len(lambda_)+1, p))
    for j in range(len(lambda_)): 
        beta_mat[j+1,:] = memmap_UAG_LM_SCAD_MCP_parallel(design_matrix=design_matrix, outcome=outcome, N=N, p=p, L_convex=L_convex, beta_0 = beta_mat[j,:], tol=tol, maxit=maxit, _lambda=lambda_[j], penalty=penalty, a=a, gamma=gamma, _dtype=_dtype, _order=_order, chunck_size=chunck_size)[1]
    return beta_mat[1:,:]



# Testing

In [4]:
import numpy as np
import matplotlib.markers as markers
import matplotlib.pyplot as plt
import timeit
from scipy.linalg import toeplitz, block_diag
from tempfile import mkdtemp
import os.path as path


np.random.seed(1)
N = 2000
SNR = 5.
true_beta = np.array([2.,-2,8,-8]+[0]*1000)
X_cov = toeplitz(.6**np.arange(true_beta.shape[0]))
mean = np.zeros(true_beta.shape[0])
X = np.random.multivariate_normal(mean, X_cov, N)
X -= np.mean(X,0).reshape(1,-1)
X /= np.std(X,0)
intercept_design_column = np.ones(N).reshape(N, 1)
X_sim = np.concatenate((intercept_design_column, X), 1)
true_sigma_sim = np.sqrt(true_beta.T@X_cov@true_beta/SNR)
true_beta_intercept = np.concatenate((np.array([1.23]), true_beta)) # here just define the intercept to be 1.23 for simulated data 
epsilon = np.random.normal(0, true_sigma_sim, N)
y_sim = X_sim@true_beta_intercept + epsilon

L_convex = 1/N*(np.linalg.eigvalsh(X_sim@X_sim.T)[-1])

filename = path.join(mkdtemp(), 'newfile.dat')
fp = np.memmap(filename, dtype='float32', mode='w+', shape=(N,1005), order="F")
fp[:] = X_sim[:]
fp.flush()


lambda_seq = np.arange(40.)/40
lambda_seq = lambda_seq[1:]
lambda_seq = lambda_seq[::-1]

# do NOT include the design matrix intercept column 
LM_beta = memmap_solution_path_LM_parallel(design_matrix=filename, N=N, p=1005, outcome=y_sim, lambda_=lambda_seq, L_convex=L_convex, beta_0 = np.ones(1), tol=1e-2, maxit=500, penalty="SCAD", a=3.7, gamma=2.,_dtype="float32", _order="F")

print(LM_beta) # to make sure strong rule runs correctly 

# %timeit solution_path_LM_strongrule(design_matrix=X_sim, outcome=y_sim, lambda_=lambda_seq, beta_0 = np.ones(1), tol=1e-2, maxit=500, penalty="SCAD", a=3.7, gamma=2., add_intercept_column=True)

[[ 1.31164792  0.05170507  0.         ...  0.          0.
   0.        ]
 [ 1.31116109  0.06873109  0.         ...  0.          0.
   0.        ]
 [ 1.31064586  0.11006317  0.         ...  0.          0.
   0.        ]
 ...
 [ 1.31048971  2.17130055 -2.18983369 ...  0.          0.
   0.        ]
 [ 1.3104897   2.16450848 -2.18883627 ...  0.          0.
   0.00863641]
 [ 1.31048972  2.17160476 -2.16255524 ...  0.         -0.03645407
   0.03595923]]


In [5]:
import matplotlib.markers as markers
import matplotlib.pyplot as plt
import timeit
from scipy.linalg import toeplitz, block_diag
from tempfile import mkdtemp
import os.path as path


np.random.seed(1)
N = 2000
SNR = 5.
true_beta = np.array([2.,-2,8,-8]+[0]*1000)
X_cov = toeplitz(.6**np.arange(true_beta.shape[0]))
mean = np.zeros(true_beta.shape[0])
X = np.random.multivariate_normal(mean, X_cov, N)
X -= np.mean(X,0).reshape(1,-1)
X /= np.std(X,0)
intercept_design_column = np.ones(N).reshape(N, 1)
X_sim = np.concatenate((intercept_design_column, X), 1)
true_sigma_sim = np.sqrt(true_beta.T@X_cov@true_beta/SNR)
true_beta_intercept = np.concatenate((np.array([1.23]), true_beta)) # here just define the intercept to be 1.23 for simulated data 
epsilon = np.random.normal(0, true_sigma_sim, N)
y_sim = X_sim@true_beta_intercept + epsilon

L_convex = 1/N*(np.linalg.eigvalsh(X_sim@X_sim.T)[-1])

filename = path.join(mkdtemp(), 'newfile.dat')
fp = np.memmap(filename, dtype='float32', mode='w+', shape=(N,1005), order="C")
fp[:] = X_sim[:]
fp.flush()


lambda_seq = np.arange(40.)/40
lambda_seq = lambda_seq[1:]
lambda_seq = lambda_seq[::-1]

# do NOT include the design matrix intercept column 
LM_beta = memmap_solution_path_LM_parallel(design_matrix=filename, N=N, p=1005, outcome=y_sim, lambda_=lambda_seq, L_convex=L_convex, beta_0 = np.ones(1), tol=1e-2, maxit=500, penalty="SCAD", a=3.7, gamma=2.,_dtype="float32", _order="C")

print(LM_beta) # to make sure strong rule runs correctly 

# %timeit solution_path_LM_strongrule(design_matrix=X_sim, outcome=y_sim, lambda_=lambda_seq, beta_0 = np.ones(1), tol=1e-2, maxit=500, penalty="SCAD", a=3.7, gamma=2., add_intercept_column=True)

[[ 1.31164787  0.05170503  0.         ...  0.          0.
   0.        ]
 [ 1.31116107  0.06873109  0.         ...  0.          0.
   0.        ]
 [ 1.31064584  0.11006316  0.         ...  0.          0.
   0.        ]
 ...
 [ 1.31048968  2.17130057 -2.18983366 ...  0.          0.
   0.        ]
 [ 1.31048968  2.1645084  -2.18883634 ...  0.          0.
   0.00863642]
 [ 1.31048968  2.1716046  -2.16255549 ...  0.         -0.03645405
   0.03595925]]
